In [1]:
#导入数据
library(tidyverse)
library(cluster)
library(ggplot2)
tv_data <- read.csv("D:\\000zyf\\Learning\\python_learn\\spider\\script\\练习\\douban_tv_filter\\DoubanTv.csv", col.names = c("ID", "Title", "Value", "Info", "Year", "Pic_url"), header = F, sep = ",")
tv_data<-tv_data%>%select(-Pic_url)

Warning message:
"package 'ggplot2' was built under R version 4.3.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
tags<-c("喜剧","爱情","悬疑", "动画","武侠","古装","家庭","犯罪","科幻","恐怖","历史","战争","动作","冒险","传记","剧情","奇幻","惊悚","灾难","歌舞","音乐")

In [3]:
tv_data %>%  filter(ID=='36369442') %>% separate(Info, c("Year1", "Area", "Type", "Director", "Actor"),sep = c(' \\ / '))

Warning message:
"Expected 5 pieces. Missing pieces filled with `NA` in 1 rows [1]."


ID,Title,Value,Year1,Area,Type,Director,Actor,Year
<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
36369442,快乐的大人,9.2,2023 / 中国大陆 / 沈月 王敬轩,NA,NA,NA,NA,2023


In [4]:
#数据里面有NA值，我们这里将其过滤掉
tv_data %>% select(Info) %>% separate(Info, c("Year1", "Area", "Type", "Director", "Actor"),sep = '/') %>% rowwise() %>% filter(any(map_lgl(c_across(everything()), is.na)))

Warning message:
"Expected 5 pieces. Missing pieces filled with `NA` in 20 rows [107, 131, 143,
146, 191, 211, 263, 288, 339, 359, 362, 379, 401, 403, 409, 461, 465, 475, 482,
486]."


Year1,Area,Type,Director,Actor
<chr>,<chr>,<chr>,<chr>,<chr>
2023,中国大陆,纪录片,肖姝,NA
2023,中国大陆,真人秀,陈少熙 何浩楠,NA
2022,中国大陆,音乐 真人秀,陈楚生 苏醒,NA
2023,中国大陆,动画 短片 奇幻,陈廖宇 於水 胡睿 杨木 刘毛宁 陈莲华 周小琳 潘斌 顾杨 刘旷 徐宁,NA
2023,中国大陆,运动 真人秀,李晨 张伟丽,NA
2023,中国大陆,喜剧 真人秀,郎昊辰 张九林,NA
2021,中国大陆,真人秀,马东 黄渤,NA
2019,中国大陆,真人秀,迪丽热巴 郑恺,NA
2023,中国大陆,王运辉,NA,NA


In [5]:
tv_data_filter<-tv_data %>% separate(Info, c("Year1", "Area", "Type", "Director", "Actor"),sep = '/') %>% na.omit() %>% select(-Year1) %>% filter(Value!=0)
tv_data_filter %>% dim()

Warning message:
"Expected 5 pieces. Missing pieces filled with `NA` in 20 rows [107, 131, 143,
146, 191, 211, 263, 288, 339, 359, 362, 379, 401, 403, 409, 461, 465, 475, 482,
486]."


[1] 457   8

In [6]:
tv_data_filter %>%
  rowwise() %>%
  filter(any(map_lgl(tags, ~str_detect(Type, .)))) %>% dim()

[1] 437   8

In [7]:
tv_data_type<-tv_data_filter %>% mutate(Type = str_trim(Type)) %>% separate(Type,c('type1','type2','type3','type4','type5','type6'),sep = ' ') %>% rowwise() %>%
  filter(all(c_across(type1:type6) %in% tags | is.na(c_across(type1:type6)))) %>% select(ID,Year,Title,type1:type6)
tv_data_value<-tv_data_filter %>% mutate(Type = str_trim(Type)) %>% separate(Type,c('type1','type2','type3','type4','type5','type6'),sep = ' ') %>% rowwise() %>%
  filter(all(c_across(type1:type6) %in% tags | is.na(c_across(type1:type6)))) %>% select(ID,Title,Year,Value)
tv_data_director<-tv_data_filter %>% mutate(Type = str_trim(Type)) %>% separate(Type,c('type1','type2','type3','type4','type5','type6'),sep = ' ') %>% rowwise() %>%
  filter(all(c_across(type1:type6) %in% tags | is.na(c_across(type1:type6)))) %>% select(ID,Title,Year,Director)
tv_data_actor<-tv_data_filter %>% mutate(Type = str_trim(Type)) %>% separate(Type,c('type1','type2','type3','type4','type5','type6'),sep = ' ') %>% rowwise() %>%
  filter(all(c_across(type1:type6) %in% tags | is.na(c_across(type1:type6)))) %>% select(ID,Title,Year,Actor)

Warning message:
"Expected 6 pieces. Missing pieces filled with `NA` in 455 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."
Warning message:
"Expected 6 pieces. Missing pieces filled with `NA` in 455 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."
Warning message:
"Expected 6 pieces. Missing pieces filled with `NA` in 455 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."
Warning message:
"Expected 6 pieces. Missing pieces filled with `NA` in 455 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."


In [8]:
tv_data_filter_final<-tv_data_filter %>% mutate(Type = str_trim(Type)) %>% separate(Type,c('type1','type2','type3','type4','type5','type6'),sep = ' ') %>% rowwise() %>%
  filter(all(c_across(type1:type6) %in% tags | is.na(c_across(type1:type6))))

Warning message:
"Expected 6 pieces. Missing pieces filled with `NA` in 455 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."


In [9]:
write.csv(tv_data_filter_final, "D:\\000zyf\\Learning\\python_learn\\spider\\script\\练习\\douban_tv_filter\\DoubanTv_filter.csv", row.names = F)

In [64]:
#绘制热力图


## 演员电视剧评分

In [15]:
tv_data_actor_value<-tv_data_filter_final %>% select(ID,Title,Year,Value,Actor)
head(tv_data_actor_value)

ID,Title,Year,Value,Actor
<int>,<chr>,<int>,<dbl>,<chr>
34874646,繁花,2023,8.6,胡歌 马伊琍
26862252,仙剑四,2024,4.2,鞠婧祎 陈哲远
35321446,大江大河之岁月如歌,2024,8.1,王凯 杨烁
35597502,祈今朝,2024,4.9,许凯 虞书欣
34948315,你也有今天,2024,7.0,陈星旭 章若楠
35588177,漫长的季节,2023,9.4,范伟 秦昊


In [60]:
#经典电视剧评分太高，
act1<-tv_data_actor_value %>% select(Value,Actor)  %>% separate(Actor, c("Actor1", "Actor2",'Actor3'),sep = ' ') %>% select(Value,Actor2) %>% rename(Actor=Actor2)
act2<-tv_data_actor_value %>% select(Value,Actor)  %>% separate(Actor, c("Actor1", "Actor2",'Actor3'),sep = ' ') %>% select(Value,Actor3) %>% rename(Actor=Actor3)
act<-bind_rows(act1,act2)
#计算演员的平均分
act %>% group_by(Actor) %>% summarise(mean_value=mean(Value,na.rm = T)) %>% arrange(desc(mean_value)) %>% head(20)

Warning message:
"Expected 3 pieces. Additional pieces discarded in 1 rows [21]."


Warning message:
"Expected 3 pieces. Additional pieces discarded in 1 rows [21]."


Actor,mean_value
<chr>,<dbl>
黄志忠,9.80
欧阳奋强,9.70
迟重瑞,9.70
陈晓旭,9.70
山新,9.60
段奕宏,9.60
王冰,9.60
皇贞季,9.60
鲍国安,9.60
